In [1]:
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib as mp
import seaborn as sb

ModuleNotFoundError: No module named 'pandas'

Firstly, we need to set our parameters. What type of distributions do we use?

For EC100, appears to be a normal distribution with mean ~ 67, SD ~ 10.5.

![images/EC100.png](images/EC100.png)

EC102 has varied a lot more YoY, for reasons I am not too sure about. 